Decomposing wide table into BCNF. 

In [1]:
import pandas as pd
import os
os.getcwd()


'/'

In [2]:

from pathlib import Path
import os
import subprocess

# Set any folder you know is in the repo
start = Path("/Users/ellawileman/Documents/fall2025/db_management/final_project_database/final_project_database")

# Walk upward until .git is found
current = start
while True:
    if ".git" in os.listdir(current):
        repo_root = current
        break
    parent = current.parent
    if parent == current:
        raise RuntimeError("Not inside a Git repo")
    current = parent

os.chdir(repo_root)
print("Repo root:", repo_root)

subprocess.run(["git", "status"]) 



Repo root: /Users/ellawileman/Documents/fall2025/db_management/final_project_database
On branch recommender-system
nothing to commit, working tree clean


CompletedProcess(args=['git', 'status'], returncode=0)

In [3]:

import subprocess

# Status
subprocess.run(["git", "status"])

# Example: list branches
subprocess.run(["git", "branch"])

# Pull latest changes
subprocess.run(["git", "pull"])



On branch recommender-system
nothing to commit, working tree clean
  checkout-feature
  main
* recommender-system


From https://github.com/mariahj990/final_project_database
 * [new branch]      recommender-system -> origin/recommender-system
There is no tracking information for the current branch.
Please specify which branch you want to merge with.
See git-pull(1) for details.

    git pull <remote> <branch>

If you wish to set tracking information for this branch you can do so with:

    git branch --set-upstream-to=origin/<branch> recommender-system



CompletedProcess(args=['git', 'pull'], returncode=1)

In [4]:
"""
import os

# List all files/folders in current directory
print(os.listdir())

# Or list them nicely, one per line
for f in os.listdir():
    print(f)
    

# Path to a file inside the repo
file_path = os.path.join(os.getcwd(), "final_project_database", "wide_book_data.csv")

"""

'\nimport os\n\n# List all files/folders in current directory\nprint(os.listdir())\n\n# Or list them nicely, one per line\nfor f in os.listdir():\n    print(f)\n    \n\n# Path to a file inside the repo\nfile_path = os.path.join(os.getcwd(), "final_project_database", "wide_book_data.csv")\n\n'

In [5]:
wide_dat = pd.read_csv("final_project_database/datasource_csvs/test_wide_with_pages.csv")
wide_dat["isbn13"] = wide_dat["isbn13"].astype("int64")

# Standardize column names with schema
wide_dat = wide_dat.rename(columns={"book_id": "bookId", "user_id": "userId", "FirstName": "firstName", "LastName": "lastName", "Username": "username"})

In [6]:
wide_dat = wide_dat.drop(['_merge', 'on_user_list'], axis=1) 
wide_dat.head()

,isbn13,description,genres,bookId,title,authors,average_rating,original_publication_year,ratings_count,image_url,userId,rating,firstName,lastName,total_copies,username,page_count
0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,1136.0,4.0,George,Roberts,4,groberts,103
1,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,1350.0,4.0,Mario,Hardin,4,mhardin,103
2,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,2276.0,4.0,Michael,Maldonado,4,mmaldonado,103
3,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,2552.0,4.0,Corey,Johnson,4,cjohnson,103
4,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,2996.0,3.0,Samuel,Dalton,4,sdalton,103


In [7]:
wide_dat.dtypes

isbn13                         int64
description                   object
genres                        object
bookId                         int64
title                         object
authors                       object
average_rating               float64
original_publication_year      int64
ratings_count                  int64
image_url                     object
userId                       float64
rating                       float64
firstName                     object
lastName                      object
total_copies                   int64
username                      object
page_count                     int64
dtype: object

R(isbn13, description, genres, book_id, title, authors, average_rating, original_publication_year, ratings_count, image_url, user_id, rating, FirstName, LastName, username, page_count, total_copies)

In [8]:
R = wide_dat

Minimal superkey (candidate key) = (book_id, user_id)

Decompose on FD #1: user_id -> FirstName, LastName, Username

In [9]:
R1 = R[["userId", "firstName", "lastName", "username"]].drop_duplicates().reset_index(drop=True)
R1 = R1.dropna(subset=['userId'])

In [10]:
duplicate_user_ids = R1["userId"].duplicated().sum()
print("Duplicate user_ids:", duplicate_user_ids)


Duplicate user_ids: 0


In [11]:
R1.head()

,userId,firstName,lastName,username
0,1136.0,George,Roberts,groberts
1,1350.0,Mario,Hardin,mhardin
2,2276.0,Michael,Maldonado,mmaldonado
3,2552.0,Corey,Johnson,cjohnson
4,2996.0,Samuel,Dalton,sdalton


In [12]:
R2 = R[["userId", "isbn13", "description", "genres", "bookId", "title", "authors", "average_rating", "original_publication_year", "ratings_count", "image_url", "rating", "total_copies", "page_count"]].drop_duplicates().reset_index(drop=True)
R2 = R2.dropna(subset=['userId'])
R2 = R2.dropna(subset=["bookId"])


In [13]:
R2.head()

,userId,isbn13,description,genres,bookId,title,authors,average_rating,original_publication_year,ratings_count,image_url,rating,total_copies,page_count
0,1136.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4.0,4,103
1,1350.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4.0,4,103
2,2276.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4.0,4,103
3,2552.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4.0,4,103
4,2996.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,3.0,4,103


Does FD #2 satisfy BCNF? 
No because book_id is not superkey. Decompose.

In [14]:
R3 = R2[["bookId", "isbn13", "description", "genres", "title", "authors", "average_rating", "original_publication_year", "ratings_count", "image_url", "total_copies", "page_count"]].drop_duplicates().reset_index(drop=True)
R3 = R3.dropna(subset=["bookId"])
R3.head()

,bookId,isbn13,description,genres,title,authors,average_rating,original_publication_year,ratings_count,image_url,total_copies,page_count
0,890,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4,103
1,1953,9780141439600,After eighteen years as a political prisoner i...,"['Classics', 'Fiction', 'Historical Fiction', ...",A Tale of Two Cities,"Charles Dickens, Richard Maxwell, Hablot Knigh...",3.81,1859,637412,https://images.gr-assets.com/books/1344922523m...,3,489
2,3636,9780385732550,Twelve-year-old Jonas lives in a seemingly ide...,"['Young Adult', 'Fiction', 'Classics', 'Dystop...","The Giver (The Giver, #1)",Lois Lowry,4.12,1993,1296825,https://images.gr-assets.com/books/1342493368m...,3,208
3,3836,9780142437230,Don Quixote has become so entranced by reading...,"['Classics', 'Fiction', 'Literature', 'Spanish...",Don Quixote,"Miguel de Cervantes Saavedra, Roberto González...",3.85,1605,134931,https://images.gr-assets.com/books/1364958765m...,1,1023
4,2165,9780684830490,Librarian's note: An alternate cover edition c...,"['Classics', 'Fiction', 'Literature', 'Novels'...",The Old Man and the Sea,Ernest Hemingway,3.73,1952,520630,https://images.gr-assets.com/books/1329189714m...,3,96


In [15]:
R4 = R2[["userId", "bookId", "rating"]].drop_duplicates(subset=["userId", "bookId"], keep="first").reset_index(drop=True)
R4 = R4.dropna(subset=['userId'])
R4 = R4.dropna(subset=["bookId"])
R4.head()

,userId,bookId,rating
0,1136.0,890,4.0
1,1350.0,890,4.0
2,2276.0,890,4.0
3,2552.0,890,4.0
4,2996.0,890,3.0


FD #3 satisfies BCNF so all decomposed relations are now in BCNF. 

In [16]:
# Adding a password column with randomly generated passwords 
import string
import random

# Function to generate a random password
""""
def generate_password(length=10):
    chars = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(chars) for _ in range(length))

# Add a password column
R1['password'] = [generate_password() for _ in range(len(R1))]
"""
R1["password"] = "password123"  # Placeholder password for all users
R1.head()

,userId,firstName,lastName,username,password
0,1136.0,George,Roberts,groberts,password123
1,1350.0,Mario,Hardin,mhardin,password123
2,2276.0,Michael,Maldonado,mmaldonado,password123
3,2552.0,Corey,Johnson,cjohnson,password123
4,2996.0,Samuel,Dalton,sdalton,password123


In [17]:
# Looking for duplicate usernames 
R1["username"].value_counts().head()
duplicate_usernames = R1[R1.duplicated(subset="username", keep=False)]

print(duplicate_usernames)
print("Number of duplicate usernames:", duplicate_usernames["username"].nunique())

Empty DataFrame
Columns: [userId, firstName, lastName, username, password]
Index: []
Number of duplicate usernames: 0


In [18]:
# Make usernames unique to uphold schema constraint
counts = {}
new_usernames = []

for u in R1["username"]:
    if u not in counts:
        counts[u] = 0
        new_usernames.append(u)
    else:
        counts[u] += 1
        new_usernames.append(f"{u}_{counts[u]}")

R1["username"] = new_usernames

In [19]:
!pip install --upgrade pip
!pip install bcrypt
!pip install swifter



In [20]:
import bcrypt
import swifter 

# Ensure passwords are strings and stripped
R1['password'] = R1['password'].astype(str).str.strip()

# Define hashing function
def hash_password(pw: str) -> str:
    pw_bytes = pw[:72].encode('utf-8')  # truncate to 72 bytes
    hashed_bytes = bcrypt.hashpw(pw_bytes, bcrypt.gensalt(rounds=8))  # lower rounds for speed
    return hashed_bytes.decode('utf-8')

# Hash all passwords in parallel
R1['password'] = R1['password'].swifter.apply(hash_password)

# Check

print(R1.head())

Pandas Apply:   0%|          | 0/7089 [00:00<?, ?it/s]

   userId firstName   lastName    username  \
0  1136.0    George    Roberts    groberts   
1  1350.0     Mario     Hardin     mhardin   
2  2276.0   Michael  Maldonado  mmaldonado   
3  2552.0     Corey    Johnson    cjohnson   
4  2996.0    Samuel     Dalton     sdalton   

                                            password  
0  $2b$08$IzRO1./.WQOe6L5qfLNvx.a9XV07cvF72qBApO9...  
1  $2b$08$qo.n6owf0f/Eg7d1asG5PuaypEfiivKFtGCr8Fw...  
2  $2b$08$CM0VR3gIPaT/gdlvS3TgVeJ4TWRhxH6KtBspGTY...  
3  $2b$08$pZtAXQT8yzqfHfHveJTB9.TtFzOkdnIsD6qWjVp...  
4  $2b$08$QqvrRBmmFn/9acPTv.qBi.wfD5iVvCqOrjfiEXK...  


In [21]:
import bcrypt

pw = "mypassword123"
hashed = bcrypt.hashpw(pw.encode('utf-8'), bcrypt.gensalt(rounds=10))
print(hashed.decode('utf-8'))


$2b$10$Sw2vrhMmNzJaiFx7C58RQeZOUE958lc2DaH0HHowJwqZHqxl5V9Kq


In [22]:
# saving the csvs

# Users Table
R1.to_csv("final_project_database/datasource_csvs/users.csv", index = False)

# Books Table
R3.to_csv("final_project_database/datasource_csvs/books.csv", index = False)

# Ratings Table 
R4.to_csv("final_project_database/datasource_csvs/ratings.csv", index = False) 

In [23]:
# Books Table
R3.to_csv("final_project_database/datasource_csvs/books.csv", index = False)

In [24]:
# only keep the entries from to_read table that have valid userId and bookId in the users and books tables

to_read = pd.read_csv("final_project_database/datasource_csvs/to_read.csv")
to_read = to_read.merge(R1[["userId"]], on="userId", how="inner")
to_read = to_read.merge(R3[["bookId"]], on="bookId", how="inner")

# Suppose history_df has userId and bookId columns
to_read = to_read[["userId", "bookId"]].drop_duplicates()
to_read.shape

(735, 2)

In [25]:
# adding has_wishliste history 
to_read["has_wishlisted"] = True

In [26]:
to_read.to_csv("final_project_database/datasource_csvs/to_read.csv", index = False)

---------------------------------------------------------------------

In [27]:
import ast
# Step 1: Convert string to actual list
df = R3 
print(type(df['genres'].iloc[0]))

# Convert stringified lists to real Python lists
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


<class 'str'>


In [28]:
# Explode directly if it's already a list
book_genre_mapping = df.explode('genres').rename(columns={'genres': 'genre'})

# Keep only book identifier and genre
book_genre_mapping = book_genre_mapping[['bookId', 'genre']]

print(book_genre_mapping.head())


   bookId               genre
0     890            Classics
0     890             Fiction
0     890              School
0     890  Historical Fiction
0     890          Literature


In [43]:
book_genre_mapping.to_csv("final_project_database/datasource_csvs/book_to_genre.csv", index = False)

In [30]:
num_unique_genres = book_genre_mapping['genre'].nunique()
print("Number of unique genres:", num_unique_genres)

Number of unique genres: 172


In [31]:
# Count how many times each genre appears
genre_counts = book_genre_mapping['genre'].value_counts()
print(genre_counts.head(10))  # Top 20 genres



genre
Fiction               71
Classics              43
Novels                40
Literature            36
Historical Fiction    25
Contemporary          22
Romance               21
Fantasy               21
Young Adult           18
Adventure             18
Name: count, dtype: int64


In [32]:
# Mapping of genres to more granular buckets
genre_to_bucket = {
    # Adult Fiction sub-buckets
    'Fiction': 'Contemporary Fiction',
    'Adult Fiction': 'Contemporary Fiction',
    'Contemporary': 'Contemporary Fiction',
    'Historical Fiction': 'Historical Fiction',
    'Romance': 'Romance / Chick Lit',
    'Chick Lit': 'Romance / Chick Lit',
    'Love': 'Romance / Chick Lit',
    'Love Story': 'Romance / Chick Lit',
    'Humor': 'Humor / Satire',
    'Comedy': 'Humor / Satire',
    'Adult Romance': 'Romance / Chick Lit',
    'Contemporary Romance': 'Romance / Chick Lit',

    # Classics & Literature sub-buckets
    'Classics': 'Classics',
    'Literature': 'Literary Fiction',
    'Novels': 'Literary Fiction',
    'Literary Fiction': 'Literary Fiction',
    'American': 'American Literature',
    'British Literature': 'European / International Literature',
    'French Literature': 'European / International Literature',
    'Spanish Literature': 'European / International Literature',
    'Irish Literature': 'European / International Literature',
    'Indian Literature': 'European / International Literature',

    # Young Adult (YA)
    'Young Adult': 'Young Adult (YA)',
    'Young Adult Contemporary': 'Young Adult (YA)',
    'Coming Of Age': 'Young Adult (YA)',
    'High School': 'Young Adult (YA)',
    'Teen': 'Young Adult (YA)',

    # Children / Middle Grade
    'Middle Grade': 'Children / Middle Grade',
    'Childrens': 'Children / Middle Grade',
    'Kids': 'Children / Middle Grade',
    'Chapter Books': 'Children / Middle Grade',
    'Picture Books': 'Children / Middle Grade',

    # Fantasy & Sci-Fi
    'Fantasy': 'Fantasy / Sci-Fi',
    'High Fantasy': 'Fantasy / Sci-Fi',
    'Epic Fantasy': 'Fantasy / Sci-Fi',
    'Urban Fantasy': 'Fantasy / Sci-Fi',
    'Science Fiction': 'Fantasy / Sci-Fi',
    'Science Fiction Fantasy': 'Fantasy / Sci-Fi',
    'Space': 'Fantasy / Sci-Fi',
    'Space Opera': 'Fantasy / Sci-Fi',
    'Magic': 'Fantasy / Sci-Fi',
    'Supernatural': 'Fantasy / Sci-Fi',
    'Speculative Fiction': 'Fantasy / Sci-Fi',
    'Time Travel': 'Fantasy / Sci-Fi',
    'Dystopia': 'Fantasy / Sci-Fi',
    'Witches': 'Fantasy / Sci-Fi',
    'Vampires': 'Fantasy / Sci-Fi',

    # Mystery / Thriller / Crime
    'Mystery': 'Mystery / Thriller',
    'Crime': 'Mystery / Thriller',
    'Mystery Thriller': 'Mystery / Thriller',
    'Legal Thriller': 'Mystery / Thriller',
    'Thriller': 'Mystery / Thriller',
    'Murder Mystery': 'Mystery / Thriller',
    'Detective': 'Mystery / Thriller',
    'Espionage': 'Mystery / Thriller',
    'Suspense': 'Mystery / Thriller',

    # Nonfiction / Biography / History
    'Historical': 'Nonfiction / History',
    'Military History': 'Nonfiction / History',
    'World History': 'Nonfiction / History',
    '19th Century': 'Nonfiction / History',
    '20th Century': 'Nonfiction / History',
    'Biography': 'Biography / Memoir',
    'Memoir': 'Biography / Memoir',
    'Autobiography': 'Biography / Memoir',
    'Essays': 'Essays / Philosophy',
    'Nonfiction': 'Nonfiction / History',
    'Philosophy': 'Essays / Philosophy',
    'Science': 'Science / Popular Science',
    'Popular Science': 'Science / Popular Science',
    'Psychology': 'Science / Popular Science',
    'Sociology': 'Science / Popular Science',
    'Economics': 'Science / Popular Science',
    'History': 'Nonfiction / History',

    # Education / School
    'School': 'Education / School',
    'Read For School': 'Education / School',
    'Teaching': 'Education / School',
    'Education': 'Education / School',

    # Miscellaneous / Lifestyle
    'Religion': 'Religion / Spirituality',
    'Christian': 'Religion / Spirituality',
    'Christian Fiction': 'Religion / Spirituality',
    'Christian Romance': 'Religion / Spirituality',
    'Atheism': 'Religion / Spirituality',
    'Faith': 'Religion / Spirituality',
    'Spirituality': 'Religion / Spirituality',
    'Jewish': 'Religion / Spirituality',
    'Cooking': 'Cooking / Food',
    'Cookbooks': 'Cooking / Food',
    'Food': 'Cooking / Food',
    'Health': 'Health / Medicine',
    'Health Care': 'Health / Medicine',
    'Medicine': 'Health / Medicine',
    'Travel': 'Travel / Geography / Nature',
    'Geography': 'Travel / Geography / Nature',
    'Environment': 'Travel / Geography / Nature',
    'Nature': 'Travel / Geography / Nature',
    'Outdoors': 'Travel / Geography / Nature',
    'Conservation': 'Travel / Geography / Nature',
    'Finance': 'Business / Finance',
    'Money': 'Business / Finance',
    'Personal Finance': 'Business / Finance',
    'Business': 'Business / Finance',
    'Self Help': 'Self Help / Personal Development',
    'Personal Development': 'Self Help / Personal Development',
    'Art': 'Arts / Music / Theatre',
    'Music': 'Arts / Music / Theatre',
    'Theatre': 'Arts / Music / Theatre',
    'Plays': 'Arts / Music / Theatre',
    'Banned Books': 'Other / Rare',
    'Unfinished': 'Other / Rare',
    'Adoption': 'Other / Rare',
    'Journalism': 'Other / Rare',
    'Sexuality': 'Other / Rare',
    'Military Fiction': 'Other / Rare',
    'Japanese Literature': 'European / International Literature',
    'Asian Literature': 'European / International Literature',
    'America': 'American Literature',
    'Europe': 'European / International Literature',
    'Africa': 'European / International Literature',
    'French Literature': 'European / International Literature',
    'Spanish Literature': 'European / International Literature',
    'Irish Literature': 'European / International Literature',
    'Indian Literature': 'European / International Literature',
}




In [33]:
# Map genres to new buckets
book_genre_mapping['bucket'] = book_genre_mapping['genre'].map(genre_to_bucket)
book_genre_mapping = book_genre_mapping.dropna(subset=['bucket'])

book_genre_mapping.head()

,bookId,genre,bucket
0,890,Classics,Classics
0,890,Fiction,Contemporary Fiction
0,890,School,Education / School
0,890,Historical Fiction,Historical Fiction
0,890,Literature,Literary Fiction


In [34]:
# Combine very small buckets into Miscellaneous / Lifestyle
small_buckets = [
    'Arts / Music / Theatre',
    'Cooking / Food',
    'Health / Medicine',
    'Business / Finance',
    'Self Help / Personal Development'
]

book_genre_mapping['bucket'] = book_genre_mapping['bucket'].replace(small_buckets, 'Miscellaneous / Lifestyle')

# Drop all rows where the bucket is 'Other / Rare'
book_genre_mapping = book_genre_mapping[book_genre_mapping['bucket'] != 'Other / Rare']


# Check the updated counts
bucket_counts = book_genre_mapping.groupby('bucket')['bookId'].nunique().sort_values(ascending=False)
print(bucket_counts)


bucket
Contemporary Fiction                   72
Literary Fiction                       49
Classics                               43
Nonfiction / History                   37
Fantasy / Sci-Fi                       27
Historical Fiction                     25
Romance / Chick Lit                    21
Young Adult (YA)                       19
Humor / Satire                         17
Children / Middle Grade                17
American Literature                    16
European / International Literature    15
Essays / Philosophy                    15
Mystery / Thriller                     14
Miscellaneous / Lifestyle              11
Education / School                     11
Biography / Memoir                     10
Science / Popular Science               7
Travel / Geography / Nature             7
Religion / Spirituality                 6
Name: bookId, dtype: int64


In [35]:
genre_category_mapping = book_genre_mapping[['genre', 'bucket']].drop_duplicates().reset_index(drop=True)
genre_category_mapping = genre_category_mapping.rename(columns={'genre': 'genreName', 'bucket': 'genreCategoryName'})
genre_category_mapping.head()

# genre_category_mapping.shape

,genreName,genreCategoryName
0,Classics,Classics
1,Fiction,Contemporary Fiction
2,School,Education / School
3,Historical Fiction,Historical Fiction
4,Literature,Literary Fiction


In [51]:
genre_category_mapping

,genreName,genreCategoryName
0,Classics,Classics
1,Fiction,Contemporary Fiction
2,School,Education / School
3,Historical Fiction,Historical Fiction
4,Literature,Literary Fiction
...,...,...
104,Indian Literature,European / International Literature
105,Health,Miscellaneous / Lifestyle
106,Health Care,Miscellaneous / Lifestyle
107,Detective,Mystery / Thriller


In [52]:
# saving this to a csv to get genre -> category mappings
genre_category_mapping.to_csv("final_project_database/datasource_csvs/genre_category.csv", index = False)

In [37]:
book_genre_mapping = book_genre_mapping[['bookId', 'bucket']].drop_duplicates().reset_index(drop=True)
book_genre_mapping = book_genre_mapping.rename(columns={'bucket': 'genreBucket'})
book_genre_mapping.head()

,bookId,genreBucket
0,890,Classics
1,890,Contemporary Fiction
2,890,Education / School
3,890,Historical Fiction
4,890,Literary Fiction


In [38]:
num_unique_genres = book_genre_mapping['genreBucket'].nunique()
print("Number of unique genres:", num_unique_genres)

Number of unique genres: 20


In [39]:
# Count how many books appear in each bucket
bucket_counts = book_genre_mapping.groupby('genreBucket')['bookId'].nunique().sort_values(ascending=False)
print(bucket_counts)


genreBucket
Contemporary Fiction                   72
Literary Fiction                       49
Classics                               43
Nonfiction / History                   37
Fantasy / Sci-Fi                       27
Historical Fiction                     25
Romance / Chick Lit                    21
Young Adult (YA)                       19
Humor / Satire                         17
Children / Middle Grade                17
American Literature                    16
European / International Literature    15
Essays / Philosophy                    15
Mystery / Thriller                     14
Miscellaneous / Lifestyle              11
Education / School                     11
Biography / Memoir                     10
Science / Popular Science               7
Travel / Geography / Nature             7
Religion / Spirituality                 6
Name: bookId, dtype: int64


In [40]:
book_genre_mapping['genreBucket'].value_counts()

genreBucket
Contemporary Fiction                   72
Literary Fiction                       49
Classics                               43
Nonfiction / History                   37
Fantasy / Sci-Fi                       27
Historical Fiction                     25
Romance / Chick Lit                    21
Young Adult (YA)                       19
Humor / Satire                         17
Children / Middle Grade                17
American Literature                    16
European / International Literature    15
Essays / Philosophy                    15
Mystery / Thriller                     14
Education / School                     11
Miscellaneous / Lifestyle              11
Biography / Memoir                     10
Travel / Geography / Nature             7
Science / Popular Science               7
Religion / Spirituality                 6
Name: count, dtype: int64

In [41]:
len(book_genre_mapping)

439

In [42]:
book_genre_mapping.head(30)

,bookId,genreBucket
0,890,Classics
1,890,Contemporary Fiction
2,890,Education / School
3,890,Historical Fiction
4,890,Literary Fiction
5,890,Young Adult (YA)
6,890,American Literature
7,1953,Classics
8,1953,Contemporary Fiction
9,1953,Historical Fiction
